In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

In [15]:
file_path = 'charity_data.csv'
charity_df = pd.read_csv(file_path)
charity_df

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [31]:
charity_names_df = charity_df.filter(['EIN', 'NAME'])
charity_names_df

,EIN,NAME
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT
...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS
34296,996012607,PTA HAWAII CONGRESS
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...


In [33]:
# Drop unneeded EIN and NAME columns from charity_df so we can perform analysis
charity_df_2 = charity_df.drop(columns=['EIN', 'NAME'])
charity_df_2

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [37]:
# Check the number of different catagorical values to see if bucketing is necessary
# Generate our categorical variable list
charity_categories = charity_df_2.dtypes[charity_df_2.dtypes == "object"].index.tolist()
# Check the number of unique values in each column
charity_df_2[charity_categories].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [39]:
# May need to bucket application type and classification
# Check the App type value counts
application_counts=charity_df_2.APPLICATION_TYPE.value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [40]:
# Replace app types with less than 500 with 'other'
replace_App = list(application_counts[application_counts < 500].index)

for app in replace_App:
    charity_df_2.APPLICATION_TYPE = charity_df_2.APPLICATION_TYPE.replace(app, 'Other')

charity_df_2.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [43]:
# Now do same process for CLASSIFICATION column
classification_counts=charity_df_2.CLASSIFICATION.value_counts()
classification_counts[:30]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
Name: CLASSIFICATION, dtype: int64

In [44]:
# Cap it at less than 100 goes to the other bin
replace_Cat = list(classification_counts[classification_counts < 100].index)

for cat in replace_Cat:
    charity_df_2.CLASSIFICATION = charity_df_2.CLASSIFICATION.replace(cat, 'Other')

charity_df_2.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [48]:
#Now bucketing is done. Can start encoding strings to numerical values
encoded_df = pd.get_dummies(charity_df_2, columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', "USE_CASE", 'ORGANIZATION', 'SPECIAL_CONSIDERATIONS', 'INCOME_AMT'])
encoded_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,SPECIAL_CONSIDERATIONS_Y,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [55]:
# Split our preprocessed data into our features and target arrays
y = encoded_df["IS_SUCCESSFUL"].values
X = encoded_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [56]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [81]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  4
#hidden_nodes_layer2 = 6

nn_7 = tf.keras.models.Sequential()

# First hidden layer
nn_7.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
#nn_6.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_7.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_7.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 4)                 200       
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 5         
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________


In [82]:
# Compile the model
nn_7.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [83]:
# Train the model
fit_model = nn_7.fit(X_train,y_train,epochs=100)

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 2s 89us/sample - loss: 20916.5377 - accuracy: 0.4941
Epoch 2/100
25724/25724 [==============================] - 2s 60us/sample - loss: 11211.3277 - accuracy: 0.5413
Epoch 3/100
25724/25724 [==============================] - 2s 60us/sample - loss: 4317.2736 - accuracy: 0.5294
Epoch 4/100
25724/25724 [==============================] - 2s 60us/sample - loss: 17455.5249 - accuracy: 0.5045
Epoch 5/100
25724/25724 [==============================] - 2s 60us/sample - loss: 13203.7591 - accuracy: 0.5364
Epoch 6/100
25724/25724 [==============================] - 2s 59us/sample - loss: 1788.5127 - accuracy: 0.5719
Epoch 7/100
25724/25724 [==============================] - 2s 59us/sample - loss: 14105.0954 - accuracy: 0.5137
Epoch 8/100
25724/25724 [==============================] - 2s 60us/sample - loss: 9210.3864 - accuracy: 0.5194
Epoch 9/100
25724/25724 [==============================] - 2s 59us/sample - loss: 17

25724/25724 [==============================] - 2s 59us/sample - loss: 12897.6750 - accuracy: 0.5569
Epoch 74/100
25724/25724 [==============================] - 2s 59us/sample - loss: 6857.2130 - accuracy: 0.5756
Epoch 75/100
25724/25724 [==============================] - 2s 59us/sample - loss: 11252.3892 - accuracy: 0.5572
Epoch 76/100
25724/25724 [==============================] - 2s 59us/sample - loss: 6896.9061 - accuracy: 0.5844
Epoch 77/100
25724/25724 [==============================] - 2s 58us/sample - loss: 12405.6949 - accuracy: 0.6160
Epoch 78/100
25724/25724 [==============================] - 2s 59us/sample - loss: 7079.8098 - accuracy: 0.6403
Epoch 79/100
25724/25724 [==============================] - 2s 61us/sample - loss: 11590.1127 - accuracy: 0.5983
Epoch 80/100
25724/25724 [==============================] - 2s 59us/sample - loss: 10966.9536 - accuracy: 0.6060
Epoch 81/100
25724/25724 [==============================] - 2s 59us/sample - loss: 925.5128 - accuracy: 0.6661
E

In [ ]:
# First attempt: two layers, 8, 5 neurons, reLU in hidden, sigmoid activation in output
# loss 0.6911, acc 0.5321
# Second attempt: one hidden layer, 8 neurons in hidden layer with reLu, sigmoid activation in output
# same exact loss and accuracy as first attempt
# Third attempt: one hidden layer 6 neurons with sigmoid activation, sigmoid activation in output
# same exact loss and accuracy as first and second attempt
# Fourth attempt: two hidden layers, 1st with 6 Neurons and reLu, 2nd with 4 neurons and sigmoid, output sigmoid
# Still getting the exact same loss and accuracy. Nothing is changing from epoch to epoch, so I must be doing something wrong
# Fifth attempt: one hidden layer, 7 neurons with relu, output has tanh
# loss 8.204, accuracy 0.4679. Still not changing much less improving from epoch to epoch
# Sixth attempt, two hidden layers, first with 8 neurons and relu, second with 6 neurons and relu, sigmoid in output
# same loss and accuracy as the first attempt. This time there was changes from epoch to epoch though
# Seventh attempt: one layer 4 neurons with relu, sigmoid output
#loss: 6862.2348, accuracy: 0.5894